In [1]:
import os
import selenium
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.webdriver.common.by import By
import xmltodict as xmld

In [2]:
kwords = pd.read_csv(os.getcwd().replace('scripts', 'data')+'/technical_buzzwords.csv')
words = pd.DataFrame(columns = ['topic', 'keywords'])
for i in range(kwords.shape[0]):
    words.loc[len(words.index)] = [kwords.iat[i,0], kwords.iat[i,1].split(", ")]


In [3]:
doi_link = 'https://api.elsevier.com/content/abstract/doi/'
pii_link = 'https://api.elsevier.com/content/abstract/pii/'

In [4]:
header = {"X-ELS-APIKey":"10ad507cd08c2ff9d4260ed62ef4cc85"}

In [5]:
def absDoi(doi):
    abstract_d = ''
    ret_d = ''
#     try:
    link = doi_link + doi +"?apiKey=10ad507cd08c2ff9d4260ed62ef4cc85"
#     print(link)
    r_doi = requests.get(link)
    ret_d = xmld.parse(r_doi.content)
    abstract_d = ret_d['abstracts-retrieval-response']['coredata']['dc:description']['abstract']['ce:para']
#     except Exception as e:
#     print("doi error")
#         print(ret_d)
#         print(str(e))
#         return []
    return abstract_d
    # r_pii = requests.get(pii_link + f'{pii}?apiKey=10ad507cd08c2ff9d4260ed62ef4cc85', {"Accept": "application/json"})

def absPii(pii):
#     pii = "S2214581816303457"
    abstract_p = ''
    ret_p = ''
#     try:
    r_pii = requests.get(pii_link + f'{pii}?apiKey=10ad507cd08c2ff9d4260ed62ef4cc85')
    ret_p = xmld.parse(r_pii.content)
    abstract_p = ret_p['abstracts-retrieval-response']['coredata']['dc:description']['abstract']['ce:para']
#     except Exception as e:
#         print(ret_p)
#         return []
    return abstract_p


#search for keywords in bastract
def find_keywords(k_list, abstract):
    ret = []
    
    old_abstract = abstract
    
#     print(k_list)
    
    try: 
        old_abstract = old_abstract['#text']
    except Exception as e:
#         print(old_abstract)
        pass
    
    orig = old_abstract

    try:
        old_abstract = old_abstract.lower().replace('-', ' ')
    except AttributeError:
        broken.append([doi, pii])
        print("BROKEN")
        print(old_abstract)
        return None
    
    for word in k_list:
        word = word.lower().replace('-', ' ')
        curr_abstract = old_abstract
        c_orig = orig
        done = False
        while(True):
            if done:
                break
            try:
                found_i = curr_abstract.index(word)

                l_b = max(0, found_i - 75)
                u_b = min(len(curr_abstract), found_i+len(word)+75)
                
                if(l_b > u_b):
                    break
                try:
                    ret.append([word,c_orig[l_b:u_b]])
                except TypeError:
                    print(c_orig)
                    print(l_b + " " + u_b)
                
#                 print(word + " "+ curr_abstract[l_b:u_b])
                
                if(u_b == len(curr_abstract)):
                    break
                
                curr_abstract = curr_abstract[found_i + len(word):]
                c_orig = c_orig[found_i + len(word):]

            except ValueError:
                done = True
                continue
    
    
    return ret#returns the specific word & context

In [43]:
papers[papers['doi'].duplicated()]

,journal_id,volume_id,year,link,doi,pii,eid,scopus_id,citations,abstract,authors,affiliations


In [32]:
papers = pd.read_csv(os.getcwd().replace('scripts', 'data')+'/all_metadata.csv')

In [42]:
for i in range(papers.shape[0]):
    au = str(papers.loc[i]['authors']).split(', ')
    aff = str(papers.loc[i]['affiliations']).split(', ')
    if len(au)!=len(aff):
        print(i)
        print(au)
        print(aff)

In [44]:
words

,topic,keywords
0,AI,"[Reinforcement Learning, Supervised Learning, ..."
1,Big Data,"[Hadoop, NoSQL, Apache Flink, Apache Kafka, Bi..."
2,Blockchain,"[Blockchain, Ethereum, Smart Contracts, Decent..."
3,IoT,"[Smart Medical Device, Industrial IoT, Home Au..."
4,VR,"[Virtual Reality, VR, Immersive Gaming, Virtua..."
5,AR,"[Google Glass, Microsoft HoloLens, AR in Marke..."
6,Machine Learning,"[Machine Learning, Scikit-Learn, TensorFlow, P..."
7,Cybersecurity,"[Cybersecurity, Ethical Hacking, Intrusion Det..."
8,Cryptocurrency,"[Cryptocurrency, Bitcoin, Ethereum, Litecoin, ..."
9,Automation,"[Automation, Industrial Automation, Home Autom..."


In [45]:
word_contexts = pd.DataFrame(columns = ['paper_id', 'category', 'sub_word', 'context'])

In [ ]:
for i in range(papers.shape[0]):
    abstract = papers.iat[i, 9]
    for j in range(words.shape[0]):
        subterms = words.iat[j, 1]
        for found in find_keywords(abstract, subterms):
            word_contexts.loc[len(word_contexts.index)] = [i, words.iat[j,0], found[0], found[1]]
